In [91]:
import pandas as pd
import numpy as np

from statistics import mean

from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import classification_report

# Import dataset

In [74]:
compressed_final = pd.read_csv("../DataFormating/compressed_final.csv")



# Setup `X`, `y` data for training / testing

In [75]:
compressed_final.columns

Index(['Year', 'Datetime', 'Stage', 'Home Team Name', 'Home Team Goals',
       'Away Team Goals', 'Away Team Name', 'Attendance',
       'Half-time Home Goals', 'Half-time Away Goals', 'Referee',
       'Assistant 1', 'Assistant 2', 'Home Team Initials',
       'Away Team Initials', 'Home Coach Name', 'Home Player 1',
       'Home Player 2', 'Home Player 3', 'Home Player 4', 'Home Player 5',
       'Home Player 6', 'Home Player 7', 'Home Player 8', 'Home Player 9',
       'Home Player 10', 'Home Player 11', 'Away Coach Name', 'Away Player 1',
       'Away Player 2', 'Away Player 3', 'Away Player 4', 'Away Player 5',
       'Away Player 6', 'Away Player 7', 'Away Player 8', 'Away Player 9',
       'Away Player 10', 'Away Player 11', 'Player 1 Stats Diff',
       'Player 2 Stats Diff', 'Player 3 Stats Diff', 'Player 4 Stats Diff',
       'Player 5 Stats Diff', 'Player 6 Stats Diff', 'Player 7 Stats Diff',
       'Player 8 Stats Diff', 'Player 9 Stats Diff', 'Player 10 Stats Diff',
     

In [76]:
X_columns = [
        'Stage', 'Attendance',
        'Referee', 'Assistant 1', 'Assistant 2', 
        'Home Team Name', 'Away Team Name', 
        'Home Coach Name', 'Away Coach Name',
    
        'Player 1 Stats Diff',
        'Player 2 Stats Diff', 'Player 3 Stats Diff', 'Player 4 Stats Diff',
        'Player 5 Stats Diff', 'Player 6 Stats Diff', 'Player 7 Stats Diff',
        'Player 8 Stats Diff', 'Player 9 Stats Diff', 'Player 10 Stats Diff',
        'Player 11 Stats Diff', 
    
        'Mean Home Team Goals', 'Mean Away Team Goals'
]

X, y = compressed_final[X_columns], []

In [77]:
for i in range(len(compressed_final)):
    home_team_goals = compressed_final.iloc[i]["Home Team Goals"]
    away_team_goals = compressed_final.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [78]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [79]:
X["Stage"] = LabelEncoder().fit_transform(X["Stage"])
X["Referee"] = LabelEncoder().fit_transform(X["Referee"])
X["Assistant 1"] = LabelEncoder().fit_transform(X["Assistant 1"])
X["Assistant 2"] = LabelEncoder().fit_transform(X["Assistant 2"])
X["Home Team Name"] = LabelEncoder().fit_transform(X["Home Team Name"])
X["Away Team Name"] = LabelEncoder().fit_transform(X["Away Team Name"])
X["Home Coach Name"] = LabelEncoder().fit_transform(X["Home Coach Name"])
X["Away Coach Name"] = LabelEncoder().fit_transform(X["Away Coach Name"])

In [81]:
len(X.columns)

22

# Fast testing

In [93]:
mean(
    cross_validate(
        RandomForestClassifier(n_estimators=100),
        X, y, cv=50
    )["test_score"]
)

0.7386666666666667

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [71]:
model = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train)
y_pred = model.predict(X_test)

In [72]:
print( classification_report(y_test, y_pred) )

             precision    recall  f1-score   support

          0       1.00      0.29      0.44        14
          1       0.58      0.88      0.70        16
          2       0.58      0.65      0.61        17

avg / total       0.71      0.62      0.59        47

